In [1]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [3]:
import pandas as pd
import requests
import json
import pprint


/Users/apabook/Desktop/ds-deployment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Testing HL Api for PULL transfer

---

## Consts

In [48]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [49]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '4c5a5e57-3c69-413c-9108-5887842bdfc4',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '4c5a5e57-3c69-413c-9108-5887842bdfc4',
 'dct:issued': '2024-11-26T08:24:12.838560',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [50]:
payload = {
    "dcat:endpointURL": context_broker + "/v2/entities"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '40c5e77b-201c-460f-9343-dccff5266249',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/entities',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '40c5e77b-201c-460f-9343-dccff5266249',
 'dct:issued': '2024-11-26T08:24:13.524596',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [51]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': 'b4999cc0-b197-4e89-9288-8bd41f083326',
 'data_service_id': '40c5e77b-201c-460f-9343-dccff5266249',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [52]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/14ac098f-3436-4c11-a4fb-4ea9a112e46f',
 'callbackId': '14ac098f-3436-4c11-a4fb-4ea9a112e46f',
 'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833'}


## Request transfer

In [53]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_pid,
  "format": "fiware+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
                     'dspace:providerPid': 'urn:uuid:1e898d78-ae7f-46a2-b278-ee8d7439efdb',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [54]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = data_space_consumer + "/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://ds-consumer:1235/14ac098f-3436-4c11-a4fb-4ea9a112e46f/data/pull/24769af9-0d78-4073-81c1-0741960a9833'}


## Use data address to get data from data space

In [55]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df["CO"]

0    {'type': 'Number', 'value': '24', 'metadata': {}}
Name: CO, dtype: object

## Suspend transfer

In [56]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
                     'dspace:providerPid': 'urn:uuid:1e898d78-ae7f-46a2-b278-ee8d7439efdb',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [57]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '56', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Restart transfer

In [58]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
                     'dspace:providerPid': 'urn:uuid:1e898d78-ae7f-46a2-b278-ee8d7439efdb',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [59]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '91', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Complete transfer

In [60]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
                     'dspace:providerPid': 'urn:uuid:1e898d78-ae7f-46a2-b278-ee8d7439efdb',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [61]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")

df = pd.read_json(url)
df

,id,type,CO,relativeHumidity,temperature
0,AirQualityUnit01,AirQualityUnit,"{'type': 'Number', 'value': '91', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}","{'type': 'Number', 'value': '0', 'metadata': {}}"


## Protocol state cannot change

In [62]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:24769af9-0d78-4073-81c1-0741960a9833',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
